In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import smtplib
import os
from typing import Dict
from IPython.display import display, Markdown

In [3]:
load_dotenv(override=True)

True

In [ ]:
INSTRUCTIONS = (
    "You are a research assistant. Given a search term, you search the web for that term and "
    "produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 "
    "words. Capture the main points. Write succintly, no need to have complete sentences or good "
    "grammar. This will be consumed by someone synthesizing a report, so its vital you capture the "
    "essence and ignore any fluff. Do not include any additional commentary other than the summary itself."
)

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
# PLANNER
NUM_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {NUM_SEARCHES} terms to query for."


class SearchItem (BaseModel):
    reason : str = Field(description="Your reasoning for why this search is important to the query")

    query : str = Field(description="The search query you will use to search the web")


class SearchPlan(BaseModel):
    searches : list[SearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="Planner",
    instructions=INSTRUCTIONS,
    model= "gpt-4o-mini",
    output_type=SearchPlan,
)

In [4]:
# EMAIL TOOL
@function_tool
def push(subject: str, message: str) -> Dict[str, str]:
    # Load environment variables inside the function
    push_mail = os.getenv("CLONE_EMAIL")
    push_password = os.getenv("CLONE_PASSWORD")    
    if not push_mail or not push_password:
        print("Error: CLONE_EMAIL or CLONE_PASSWORD not found in environment variables")
        print("Please create a .env file with CLONE_EMAIL and CLONE_PASSWORD")
        return
    try:
        mail=smtplib.SMTP("smtp.gmail.com")
        mail.starttls()
        mail.login(user=push_mail, password=push_password)
        mail.sendmail(
            from_addr=push_mail, 
            to_addrs="rumesefia@gmail.com", 
            msg=f"Subject:{subject}\n\n{message}"
            )
        mail.quit()
        status = "Email sent successfully"
    except Exception as e:
        status = "Error sending email: {e}"
    finally:
        return {"status": status}

In [5]:
push

FunctionTool(name='push', description='', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'message': {'title': 'Message', 'type': 'string'}}, 'required': ['subject', 'message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025D0B838E00>, strict_json_schema=True, is_enabled=True)

In [8]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    tools=[push],
    handoff_description= "Send a formatted email of the report"
)

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
    handoff_description="Write a detailed report based on the research findings"
)
